In [ ]:
import { load } from "dotenv";
const env = await load({
  envPath:'./.env.local',
  export: true
});

const process = {
    env
}

const chatOptions = {
  openAIApiKey: process.env.OPENAI_API_KEY,
  temperature: 1.5,
  model: "deepseek-chat",
  configuration: {
    baseURL: "https://api.deepseek.com",
  },
  azureOpenAIBasePath: "https://api.deepseek.com",
}

console.log(process.env)

langchain 的核心接口 `Runnable`，是一个流式接口，接口上有几个常用方法
- `invoke` 基础调用
- `batch` 批量调用
- `stream` 调用并以流式返回
- `streamLog` 将多个 `Runnable` 串联起来
- `pipe` 将多个 `Runnable` 串联起来


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatModel = new ChatOpenAI(chatOptions);

await chatModel.invoke([
    new HumanMessage("Tell me a joke")
])

const outputPrase = new StringOutputParser()

const simpleChain = chatModel.pipe(outputPrase)

await simpleChain.invoke([new HumanMessage("Tell me a joke")])


批量调用

In [ ]:
await simpleChain.batch([
  [ new HumanMessage("Tell me a joke") ],
  [ new HumanMessage("Hi, Who are you?") ],
])

流式输出

In [ ]:
const stream = await simpleChain.stream([
  new HumanMessage("Tell me a joke")
])

for await (const chunk of stream){
 console.log(chunk)
}

## fallback

回退机制

任何一个 Runnable 都可以设置回退机制，当调用失败时，会调用回退机制

In [ ]:
import { ChatOpenAI } from "@langchain/openai"

// 模拟一个失败的情况
const fakeLLM = new ChatOpenAI({
  azureOpenAIApiKey: "123",
  maxRetries: 0,
})

// 回退到另一个模型
const realLLM = new ChatOpenAI(chatOptions)
// 设置回退机制
const llmWithFallback = fakeLLM.withFallbacks({
  fallbacks: [realLLM],
})

await llmWithFallback.invoke("你好")
